In [ ]:
# Alfa, beta og gamma, samt halveringstid kalkulator versjon 1
"""
Fysikkmester GUI – interaktivt verktøy for kjernereaksjoner og halveringstid.
- Fanen "Henfall": skriv inn A og Z, velg strålingstype (alfa/β-/β+/γ), få nytt grunnstoff/isotop og forklaring.
- Fanen "Halveringstid": velg isotop fra nedtrekk, oppgi antall halveringer og startmasse -> gjenstående masse + tid.
- Valgfritt: Integrasjon med 'radioactivedecay' (hvis installert) for kjedediagram og eksakte halveringstider.

Avhenger kun av standardbibliotek + Tkinter. 'radioactivedecay' er valgfritt.
"""

import math
import tkinter as tk
from tkinter import ttk, messagebox

# ---------------------------------------------------
# Valgfri integrasjon med radioactivedecay
# ---------------------------------------------------
RD_AVAILABLE = False
try:
    import radioactivedecay as rd  # type: ignore
    RD_AVAILABLE = True
except Exception:
    RD_AVAILABLE = False

# ---------------------------------------------------
# Periodetabell (Z -> symbol) og omvendt
# ---------------------------------------------------
ELEMENT_SYMBOLS = [
    None,
    "H","He","Li","Be","B","C","N","O","F","Ne",
    "Na","Mg","Al","Si","P","S","Cl","Ar","K","Ca",
    "Sc","Ti","V","Cr","Mn","Fe","Co","Ni","Cu","Zn",
    "Ga","Ge","As","Se","Br","Kr","Rb","Sr","Y","Zr",
    "Nb","Mo","Tc","Ru","Rh","Pd","Ag","Cd","In","Sn",
    "Sb","Te","I","Xe","Cs","Ba","La","Ce","Pr","Nd",
    "Pm","Sm","Eu","Gd","Tb","Dy","Ho","Er","Tm","Yb",
    "Lu","Hf","Ta","W","Re","Os","Ir","Pt","Au","Hg",
    "Tl","Pb","Bi","Po","At","Rn","Fr","Ra","Ac","Th",
    "Pa","U","Np","Pu","Am","Cm","Bk","Cf","Es","Fm",
    "Md","No","Lr","Rf","Db","Sg","Bh","Hs","Mt","Ds",
    "Rg","Cn","Nh","Fl","Mc","Lv","Ts","Og"
]
SYMBOL_TO_Z = {sym: z for z, sym in enumerate(ELEMENT_SYMBOLS) if sym}

# ---------------------------------------------------
# Partikler
# ---------------------------------------------------
class Partikkel:
    def __init__(self, navn, A, Z, symbol):
        self.navn = navn
        self._A = A
        self._Z = Z
        self.symbol = symbol
    def A(self): return self._A
    def Z(self): return self._Z

PROTON  = Partikkel("proton", 1, +1, "p")
NEUTRON = Partikkel("nøytron", 1,  0, "n")
ALFA    = Partikkel("alfa",    4, +2, "α")
E_M     = Partikkel("elektron (beta-)", 0, -1, "e-")
E_P     = Partikkel("positron (beta+)", 0, +1, "e+")
GAMMA   = Partikkel("gamma",   0,  0, "γ")
NU_E    = Partikkel("elektron-nøytrino", 0, 0, "ν_e")
A_NU_E  = Partikkel("anti-elektron-nøytrino", 0, 0, "ν̄_e")

# ---------------------------------------------------
# Henfallsregler
# ---------------------------------------------------
def z_to_symbol(Z: int) -> str:
    return ELEMENT_SYMBOLS[Z] if 1 <= Z < len(ELEMENT_SYMBOLS) else "?"

def make_isotope(Z: int, A: int) -> str:
    if A <= 0 or Z < 0 or Z >= len(ELEMENT_SYMBOLS):
        raise ValueError(f"Ugyldig kjerne (A={A}, Z={Z}).")
    sym = z_to_symbol(Z)
    if sym == "?":
        raise ValueError(f"Ukjent grunnstoff for Z={Z}.")
    return f"{sym}-{A}"

def do_alpha(A: int, Z: int):
    if A < 5 or Z < 3:
        raise ValueError("Alfa-henfall krever A≥5 og Z≥3.")
    A2, Z2 = A - 4, Z - 2
    isotop = make_isotope(Z2, A2)
    forklaring = (f"Alfa (α): {z_to_symbol(Z)}-{A} → {isotop} + α\n"
                  f"A: {A}→{A2} (−4), Z: {Z}→{Z2} (−2), N: {A-Z}→{A2-Z2} (−2).")
    return isotop, forklaring

def do_beta_minus(A: int, Z: int):
    A2, Z2 = A, Z + 1
    isotop = make_isotope(Z2, A2)
    forklaring = (f"Beta- (β⁻): {z_to_symbol(Z)}-{A} → {isotop} + e⁻ + ν̄ₑ\n"
                  f"A uendret, Z: {Z}→{Z2} (+1), N: {A-Z}→{A2-Z2} (−1).")
    return isotop, forklaring

def do_beta_plus(A: int, Z: int):
    if Z < 1:
        raise ValueError("Beta+ krever Z≥1.")
    A2, Z2 = A, Z - 1
    isotop = make_isotope(Z2, A2)
    forklaring = (f"Beta+ (β⁺): {z_to_symbol(Z)}-{A} → {isotop} + e⁺ + νₑ\n"
                  f"A uendret, Z: {Z}→{Z2} (−1), N: {A-Z}→{A2-Z2} (+1).\n"
                  f"*Merk:* β⁺ krever energioverskudd eller elektroninnfangning i praksis.")
    return isotop, forklaring

def do_gamma(A: int, Z: int):
    isotop = make_isotope(Z, A)
    forklaring = (f"Gamma (γ): {z_to_symbol(Z)}-{A}* → {isotop} + γ\n"
                  f"A, Z, N uendret (de-eksitasjon).")
    return isotop, forklaring

# ---------------------------------------------------
# Halveringstid-data (fallback) i sekunder
# Oppdateres automatisk dersom radioactivedecay finnes.
# ---------------------------------------------------
FALLBACK_T12_S = {
    "C-14": 5730 * 365.25 * 24*3600,
    "U-238": 4.468e9 * 365.25 * 24*3600,
    "Co-60": 5.2714 * 365.25 * 24*3600,
    "I-131": 8.0197 * 24*3600,
    "Cs-137": 30.17 * 365.25 * 24*3600,
    "Rn-222": 3.8235 * 24*3600,
    "K-40": 1.251e9 * 365.25 * 24*3600,
    "Sr-90": 28.90 * 365.25 * 24*3600,
    "Po-210": 138.376 * 24*3600,
    "Ra-226": 1600 * 365.25 * 24*3600,
    "Na-22": 2.6018 * 365.25 * 24*3600,
    "Th-232": 1.405e10 * 365.25 * 24*3600,
}

ISOTOP_LISTE = list(FALLBACK_T12_S.keys())  # rekkefølge i nedtrekk

def rd_half_life_readable(nuclide: str) -> str:
    """Hent halveringstid som lesbar streng fra radioactivedecay om mulig."""
    if RD_AVAILABLE:
        try:
            return rd.Nuclide(nuclide).half_life("readable")
        except Exception:
            pass
    # Fallback: grov lesbar formattering
    secs = FALLBACK_T12_S.get(nuclide)
    if secs is None:
        return "ukjent"
    return human_readable_time(secs)

def rd_half_life_seconds(nuclide: str) -> float:
    """Halveringstid i sekunder (radioactivedecay hvis mulig, ellers fallback)."""
    if RD_AVAILABLE:
        try:
            return float(rd.Nuclide(nuclide).half_life("s"))
        except Exception:
            pass
    return float(FALLBACK_T12_S[nuclide])

def human_readable_time(seconds: float) -> str:
    """Gjør sekunder om til en kort lesbar streng."""
    units = [
        ("y", 365.25*86400),
        ("d", 86400),
        ("h", 3600),
        ("min", 60),
        ("s", 1),
    ]
    remaining = seconds
    parts = []
    for name, size in units:
        if remaining >= size:
            val = int(remaining // size)
            remaining -= val * size
            parts.append(f"{val} {name}")
        if len(parts) >= 2 and name != "s":  # hold det kort (to ledd)
            break
    if not parts:
        return f"{seconds:.3g} s"
    return " ".join(parts)

# ---------------------------------------------------
# GUI
# ---------------------------------------------------
class FysikkmesterApp(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Fysikkmester – Kjernereaksjoner og halveringstid")
        self.geometry("800x600")
        self.minsize(760, 560)

        nb = ttk.Notebook(self)
        self.tab_henfall = ttk.Frame(nb)
        self.tab_halv = ttk.Frame(nb)
        nb.add(self.tab_henfall, text="Henfall (A, Z → ny isotop)")
        nb.add(self.tab_halv, text="Halveringstid (nedtrekk)")
        nb.pack(fill="both", expand=True)

        self.build_henfall_tab()
        self.build_halvering_tab()

    # ---------------- Henfall-fanen ----------------
    def build_henfall_tab(self):
        frm = ttk.Frame(self.tab_henfall, padding=12)
        frm.pack(fill="both", expand=True)

        # Inndata A og Z
        box = ttk.LabelFrame(frm, text="Inndata for kjerne")
        box.pack(fill="x", pady=6)

        ttk.Label(box, text="Nukleontall A:").grid(row=0, column=0, sticky="w", padx=6, pady=6)
        self.var_A = tk.StringVar(value="238")
        spin_A = ttk.Spinbox(box, from_=1, to=1000, textvariable=self.var_A, width=8)
        spin_A.grid(row=0, column=1, padx=6, pady=6)

        ttk.Label(box, text="Protontall Z (atomnummer):").grid(row=0, column=2, sticky="w", padx=6, pady=6)
        self.var_Z = tk.StringVar(value="92")
        spin_Z = ttk.Spinbox(box, from_=0, to=118, textvariable=self.var_Z, width=8, command=self.update_symbol)
        spin_Z.grid(row=0, column=3, padx=6, pady=6)

        ttk.Label(box, text="Element:").grid(row=0, column=4, sticky="e", padx=6, pady=6)
        self.lbl_symbol = ttk.Label(box, text="Uran (U)")
        self.lbl_symbol.grid(row=0, column=5, sticky="w", padx=6, pady=6)

        # Valg av strålingstype
        box2 = ttk.LabelFrame(frm, text="Velg strålingstype")
        box2.pack(fill="x", pady=6)
        self.var_mode = tk.StringVar(value="alfa")
        ttk.Radiobutton(box2, text="Alfa (α)", variable=self.var_mode, value="alfa").grid(row=0, column=0, padx=6, pady=6, sticky="w")
        ttk.Radiobutton(box2, text="Beta- (β⁻)", variable=self.var_mode, value="beta-").grid(row=0, column=1, padx=6, pady=6, sticky="w")
        ttk.Radiobutton(box2, text="Beta+ (β⁺)", variable=self.var_mode, value="beta+").grid(row=0, column=2, padx=6, pady=6, sticky="w")
        ttk.Radiobutton(box2, text="Gamma (γ)", variable=self.var_mode, value="gamma").grid(row=0, column=3, padx=6, pady=6, sticky="w")

        btn = ttk.Button(frm, text="Beregn henfall", command=self.calc_decay)
        btn.pack(pady=8)

        self.txt_out = tk.Text(frm, height=14, wrap="word")
        self.txt_out.pack(fill="both", expand=True, pady=6)
        self.txt_out.configure(state="disabled")

        self.update_symbol()

    def update_symbol(self):
        try:
            Z = int(self.var_Z.get())
            sym = z_to_symbol(Z)
            navn = {
                "H":"Hydrogen","He":"Helium","Li":"Litium","Be":"Beryllium","B":"Bor","C":"Karbon","N":"Nitrogen","O":"Oksygen","F":"Fluor","Ne":"Neon",
                "Na":"Natrium","Mg":"Magnesium","Al":"Aluminium","Si":"Silisium","P":"Fosfor","S":"Svovel","Cl":"Klor","Ar":"Argon","K":"Kalium","Ca":"Kalsium",
                "Fe":"Jern","Ni":"Nikkel","Cu":"Kopper","Zn":"Sink","Ga":"Gallium","Ge":"Germanium","As":"Arsen","Se":"Selen","Br":"Brom","Kr":"Krypton",
                "Rb":"Rubidium","Sr":"Strontium","Y":"Yttrium","Zr":"Zirkonium","Nb":"Niob","Mo":"Molybden","Tc":"Teknetium","Ru":"Ruthenium","Rh":"Rodium","Pd":"Palladium",
                "Ag":"Sølv","Cd":"Kadmium","In":"Indium","Sn":"Tinn","Sb":"Antimon","Te":"Tellur","I":"Jod","Xe":"Xenon","Cs":"Cesium","Ba":"Barium",
                "La":"Lantan","Ce":"Cerium","Pr":"Praseodym","Nd":"Neodym","Pm":"Prometium","Sm":"Samarium","Eu":"Europium","Gd":"Gadolinium","Tb":"Terbium","Dy":"Dysprosium",
                "Ho":"Holmium","Er":"Erbium","Tm":"Tulium","Yb":"Ytterbium","Lu":"Lutetium","Hf":"Hafnium","Ta":"Tantal","W":"Wolfram","Re":"Rhenium","Os":"Osmium",
                "Ir":"Iridium","Pt":"Platina","Au":"Gull","Hg":"Kvikksølv","Tl":"Tallium","Pb":"Bly","Bi":"Vismut","Po":"Polonium","At":"Astat","Rn":"Radon",
                "Fr":"Francium","Ra":"Radium","Ac":"Aktinium","Th":"Thorium","Pa":"Protaktinium","U":"Uran","Np":"Neptunium","Pu":"Plutonium"
            }.get(sym, sym)
            self.lbl_symbol.config(text=f"{navn} ({sym})" if sym!="?" else "Ukjent")
        except Exception:
            self.lbl_symbol.config(text="Ukjent")

    def calc_decay(self):
        self.txt_out.configure(state="normal")
        self.txt_out.delete("1.0", "end")
        try:
            A = int(self.var_A.get())
            Z = int(self.var_Z.get())
            if A <= 0 or Z < 0 or Z > A:
                raise ValueError("Sjekk at A>0 og 0 ≤ Z ≤ A.")
            mode = self.var_mode.get()
            if mode == "alfa":
                iso, info = do_alpha(A, Z)
                part = " + α"
            elif mode == "beta-":
                iso, info = do_beta_minus(A, Z)
                part = " + e⁻ + ν̄ₑ"
            elif mode == "beta+":
                iso, info = do_beta_plus(A, Z)
                part = " + e⁺ + νₑ"
            else:
                iso, info = do_gamma(A, Z)
                part = " + γ"

            start = f"{z_to_symbol(Z)}-{A}"
            likn = f"Reaksjon: {start} → {iso}{part}"
            self.txt_out.insert("end", likn + "\n" + info + "\n")
            self.txt_out.insert("end", f"Ny isotop/grunnstoff: {iso}\n")
            self.txt_out.insert("end", f"N (nøytroner): {A-Z} → {int(iso.split('-')[1]) - SYMBOL_TO_Z[iso.split('-')[0]]}\n")
        except Exception as e:
            messagebox.showerror("Feil i henfall", str(e))
        finally:
            self.txt_out.configure(state="disabled")

    # ---------------- Halveringstid-fanen ----------------
    def build_halvering_tab(self):
        frm = ttk.Frame(self.tab_halv, padding=12)
        frm.pack(fill="both", expand=True)

        line1 = ttk.Frame(frm)
        line1.pack(fill="x", pady=4)

        ttk.Label(line1, text="Velg isotop:").pack(side="left")
        self.iso_var = tk.StringVar(value=ISOTOP_LISTE[0])
        self.iso_combo = ttk.Combobox(line1, textvariable=self.iso_var, values=ISOTOP_LISTE, state="readonly", width=12)
        self.iso_combo.pack(side="left", padx=6)
        self.iso_combo.bind("<<ComboboxSelected>>", lambda e: self.update_halflife_label())

        ttk.Label(line1, text="Halveringstid:").pack(side="left", padx=(16, 4))
        self.lbl_t12 = ttk.Label(line1, text="")
        self.lbl_t12.pack(side="left")
        self.update_halflife_label()

        if RD_AVAILABLE:
            btn_chain = ttk.Button(line1, text="Vis kjedediagram", command=self.show_chain_plot)
            btn_chain.pack(side="right")

        line2 = ttk.Frame(frm)
        line2.pack(fill="x", pady=4)
        ttk.Label(line2, text="Startmasse:").grid(row=0, column=0, sticky="w")
        self.var_m = tk.StringVar(value="1.0")
        ttk.Entry(line2, textvariable=self.var_m, width=10).grid(row=0, column=1, padx=6)
        self.unit_var = tk.StringVar(value="kg")
        ttk.Combobox(line2, textvariable=self.unit_var, values=["kg","g","mg"], state="readonly", width=5)\
            .grid(row=0, column=2)

        ttk.Label(line2, text="Antall halveringer n:").grid(row=0, column=3, padx=(16,4))
        self.var_n = tk.StringVar(value="2")
        ttk.Spinbox(line2, from_=0, to=200, textvariable=self.var_n, width=6).grid(row=0, column=4)

        btn_calc = ttk.Button(frm, text="Beregn gjenstående masse", command=self.calc_mass_after_halflives)
        btn_calc.pack(pady=8)

        self.txt_halv = tk.Text(frm, height=16, wrap="word")
        self.txt_halv.pack(fill="both", expand=True, pady=4)
        self.txt_halv.configure(state="disabled")

        # Info
        info = ttk.Label(frm, foreground="#555",
                         text="Formel: N = N0 · 2^(-n). Forløpt tid = n · T1/2. (Arealutnyttelse: viser også T1/2 fra database hvis tilgjengelig.)")
        info.pack(anchor="w", pady=(2,0))

    def update_halflife_label(self):
        iso = self.iso_var.get()
        t12_str = rd_half_life_readable(iso)
        self.lbl_t12.config(text=t12_str)

    def calc_mass_after_halflives(self):
        self.txt_halv.configure(state="normal")
        self.txt_halv.delete("1.0", "end")
        try:
            iso = self.iso_var.get()
            n = float(self.var_n.get())
            m0 = float(self.var_m.get())
            unit = self.unit_var.get()
            # Konverter masse til kg internt
            factor = {"kg":1.0, "g":1e-3, "mg":1e-6}[unit]
            m0kg = m0 * factor
            frac = 2.0 ** (-n)   # N/N0
            mkg = m0kg * frac
            # tilbake til valgt enhet
            m_out = mkg / factor

            # Tid som har gått
            t12_s = rd_half_life_seconds(iso)
            t_elapsed_s = n * t12_s
            t_elapsed_str = human_readable_time(t_elapsed_s)
            t12_str = rd_half_life_readable(iso)

            self.txt_halv.insert("end", f"Isotop: {iso}\n")
            self.txt_halv.insert("end", f"Halveringstid T1/2: {t12_str}\n")
            self.txt_halv.insert("end", f"Antall halveringer n: {n:g}\n")
            self.txt_halv.insert("end", f"Startmasse: {m0:g} {unit}\n\n")
            self.txt_halv.insert("end", f"Gjenstående fraksjon: {frac:.6f}\n")
            self.txt_halv.insert("end", f"Gjenstående masse: {m_out:.6g} {unit}\n")
            self.txt_halv.insert("end", f"Forløpt tid: ~ {t_elapsed_str}\n")
        except Exception as e:
            messagebox.showerror("Feil i halveringsberegning", str(e))
        finally:
            self.txt_halv.configure(state="disabled")

    def show_chain_plot(self):
        if not RD_AVAILABLE:
            messagebox.showwarning("radioactivedecay mangler",
                                   "Installer pakken for å vise kjedediagram:\n\npip install radioactivedecay")
            return
        try:
            iso = self.iso_var.get()
            # Tegn kjedediagram i eget Matplotlib-vindu
            nuc = rd.Nuclide(iso)
            fig, ax = nuc.plot(label_pos=0.6)
            ax.set_title(f"Henfallskjede for {iso}")
            # Forsinket import for å unngå krav når RD ikke er tilgjengelig
            import matplotlib.pyplot as plt  # type: ignore
            plt.show()
        except Exception as e:
            messagebox.showerror("Kjedediagram", str(e))

if __name__ == "__main__":
    app = FysikkmesterApp()
    app.mainloop()

In [2]:
# versjon 1,5
"""
Fysikkmester GUI – interaktivt verktøy for kjernereaksjoner og halveringstid.
- Fane "Henfall":
    * Skriv inn isotopnavn (f.eks. U-238 / 238U / C-14) ELLER A og Z.
    * Velg strålingstype (alfa/β-/β+/γ) -> nytt isotop + forklaring.
- Fane "Halveringstid":
    * Velg isotop fra nedtrekk og oppgi startmasse m₀.
    * Modus 1: Inndata er antall halveringer n -> gjenstående masse + medgått tid.
    * Modus 2: Inndata er tid t (med enhet) -> beregnet n + gjenstående masse.
- Valgfritt: Integrasjon med 'radioactivedecay' (hvis installert) for eksakte halveringstider.

Avhenger kun av standardbibliotek + Tkinter. 'radioactivedecay' er valgfritt.
"""

import math
import re
import tkinter as tk
from tkinter import ttk, messagebox

# ---------------------------------------------------
# Valgfri integrasjon med radioactivedecay
# ---------------------------------------------------
RD_AVAILABLE = False
try:
    import radioactivedecay as rd  # type: ignore
    RD_AVAILABLE = True
except Exception:
    RD_AVAILABLE = False

# ---------------------------------------------------
# Periodetabell (Z -> symbol) og omvendt
# ---------------------------------------------------
ELEMENT_SYMBOLS = [
    None,
    "H","He","Li","Be","B","C","N","O","F","Ne",
    "Na","Mg","Al","Si","P","S","Cl","Ar","K","Ca",
    "Sc","Ti","V","Cr","Mn","Fe","Co","Ni","Cu","Zn",
    "Ga","Ge","As","Se","Br","Kr","Rb","Sr","Y","Zr",
    "Nb","Mo","Tc","Ru","Rh","Pd","Ag","Cd","In","Sn",
    "Sb","Te","I","Xe","Cs","Ba","La","Ce","Pr","Nd",
    "Pm","Sm","Eu","Gd","Tb","Dy","Ho","Er","Tm","Yb",
    "Lu","Hf","Ta","W","Re","Os","Ir","Pt","Au","Hg",
    "Tl","Pb","Bi","Po","At","Rn","Fr","Ra","Ac","Th",
    "Pa","U","Np","Pu","Am","Cm","Bk","Cf","Es","Fm",
    "Md","No","Lr","Rf","Db","Sg","Bh","Hs","Mt","Ds",
    "Rg","Cn","Nh","Fl","Mc","Lv","Ts","Og"
]
SYMBOL_TO_Z = {sym: z for z, sym in enumerate(ELEMENT_SYMBOLS) if sym}
SYMBOL_UPPER_TO_CANON = {sym.upper(): sym for sym in SYMBOL_TO_Z.keys()}

# ---------------------------------------------------
# Partikler
# ---------------------------------------------------
class Partikkel:
    def __init__(self, navn, A, Z, symbol):
        self.navn = navn
        self._A = A
        self._Z = Z
        self.symbol = symbol
    def A(self): return self._A
    def Z(self): return self._Z

PROTON  = Partikkel("proton", 1, +1, "p")
NEUTRON = Partikkel("nøytron", 1,  0, "n")
ALFA    = Partikkel("alfa",    4, +2, "α")
E_M     = Partikkel("elektron (beta-)", 0, -1, "e-")
E_P     = Partikkel("positron (beta+)", 0, +1, "e+")
GAMMA   = Partikkel("gamma",   0,  0, "γ")
NU_E    = Partikkel("elektron-nøytrino", 0, 0, "ν_e")
A_NU_E  = Partikkel("anti-elektron-nøytrino", 0, 0, "ν̄_e")

# ---------------------------------------------------
# Henfallsregler og hjelpere
# ---------------------------------------------------
def z_to_symbol(Z: int) -> str:
    return ELEMENT_SYMBOLS[Z] if 1 <= Z < len(ELEMENT_SYMBOLS) else "?"

def make_isotope(Z: int, A: int) -> str:
    if A <= 0 or Z < 0 or Z >= len(ELEMENT_SYMBOLS):
        raise ValueError(f"Ugyldig kjerne (A={A}, Z={Z}).")
    sym = z_to_symbol(Z)
    if sym == "?":
        raise ValueError(f"Ukjent grunnstoff for Z={Z}.")
    return f"{sym}-{A}"

def do_alpha(A: int, Z: int):
    if A < 5 or Z < 3:
        raise ValueError("Alfa-henfall krever A≥5 og Z≥3.")
    A2, Z2 = A - 4, Z - 2
    isotop = make_isotope(Z2, A2)
    forklaring = (f"Alfa (α): {z_to_symbol(Z)}-{A} → {isotop} + α\n"
                  f"A: {A}→{A2} (−4), Z: {Z}→{Z2} (−2), N: {A-Z}→{A2-Z2} (−2).")
    return isotop, forklaring

def do_beta_minus(A: int, Z: int):
    A2, Z2 = A, Z + 1
    isotop = make_isotope(Z2, A2)
    forklaring = (f"Beta- (β⁻): {z_to_symbol(Z)}-{A} → {isotop} + e⁻ + ν̄ₑ\n"
                  f"A uendret, Z: {Z}→{Z2} (+1), N: {A-Z}→{A2-Z2} (−1).")
    return isotop, forklaring

def do_beta_plus(A: int, Z: int):
    if Z < 1:
        raise ValueError("Beta+ krever Z≥1.")
    A2, Z2 = A, Z - 1
    isotop = make_isotope(Z2, A2)
    forklaring = (f"Beta+ (β⁺): {z_to_symbol(Z)}-{A} → {isotop} + e⁺ + νₑ\n"
                  f"A uendret, Z: {Z}→{Z2} (−1), N: {A-Z}→{A2-Z2} (+1).\n"
                  f"*Merk:* β⁺ krever energioverskudd eller elektroninnfangning i praksis.")
    return isotop, forklaring

def do_gamma(A: int, Z: int):
    isotop = make_isotope(Z, A)
    forklaring = (f"Gamma (γ): {z_to_symbol(Z)}-{A}* → {isotop} + γ\n"
                  f"A, Z, N uendret (de-eksitasjon).")
    return isotop, forklaring

# ---------------------------------------------------
# Isotop-parsing (brukerskrivbar: "U-238", "238U", "C14", "na-22", osv.)
# ---------------------------------------------------
ISO_PATTERNS = [
    re.compile(r"^\s*([A-Za-z]{1,3})\s*[-\s]?\s*(\d{1,3})\s*$"),  # U-238 / U 238 / U238 / Na-22
    re.compile(r"^\s*(\d{1,3})\s*([A-Za-z]{1,3})\s*$"),           # 238U / 14C
]

def normaliser_symbol(sym_inn: str) -> str:
    """Case-uavhengig symbol-normalisering til kanonisk kjemisk symbol."""
    key = sym_inn.strip().upper()
    if key in SYMBOL_UPPER_TO_CANON:
        return SYMBOL_UPPER_TO_CANON[key]
    raise ValueError(f"Ukjent kjemisk symbol: '{sym_inn}'.")

def parse_isotopnavn(tekst: str):
    """
    Returnerer (A, Z) fra en streng som 'U-238', '238U', 'C-14', 'na-22' osv.
    Kaster ValueError ved feil format.
    """
    t = (tekst or "").strip()
    if not t:
        raise ValueError("Tom isotopstreng.")
    for pat in ISO_PATTERNS:
        m = pat.match(t)
        if m:
            if pat is ISO_PATTERNS[0]:
                sym_raw, A_str = m.groups()
            else:
                A_str, sym_raw = m.groups()
            sym = normaliser_symbol(sym_raw)
            Z = SYMBOL_TO_Z[sym]
            A = int(A_str)
            if A <= 0 or Z <= 0 or Z > A:
                raise ValueError(f"Ugyldig isotop: {sym}-{A}.")
            return A, Z
    raise ValueError("Ugyldig isotopformat. Bruk f.eks. 'U-238' eller '238U'.")

# ---------------------------------------------------
# Halveringstid-data (fallback) i sekunder
# Oppdateres automatisk dersom radioactivedecay finnes.
# ---------------------------------------------------
FALLBACK_T12_S = {
    "C-14": 5730 * 365.25 * 24*3600,
    "U-238": 4.468e9 * 365.25 * 24*3600,
    "Co-60": 5.2714 * 365.25 * 24*3600,
    "I-131": 8.0197 * 24*3600,
    "Cs-137": 30.17 * 365.25 * 24*3600,
    "Rn-222": 3.8235 * 24*3600,
    "K-40": 1.251e9 * 365.25 * 24*3600,
    "Sr-90": 28.90 * 365.25 * 24*3600,
    "Po-210": 138.376 * 24*3600,
    "Ra-226": 1600 * 365.25 * 24*3600,
    "Na-22": 2.6018 * 365.25 * 24*3600,
    "Th-232": 1.405e10 * 365.25 * 24*3600,
}
ISOTOP_LISTE = list(FALLBACK_T12_S.keys())  # rekkefølge i nedtrekk

def rd_half_life_readable(nuclide: str) -> str:
    """Hent halveringstid som lesbar streng fra radioactivedecay om mulig."""
    if RD_AVAILABLE:
        try:
            return rd.Nuclide(nuclide).half_life("readable")  # type: ignore[attr-defined]
        except Exception:
            pass
    # Fallback: grov lesbar formattering
    secs = FALLBACK_T12_S.get(nuclide)
    if secs is None:
        return "ukjent"
    return human_readable_time(secs)

def rd_half_life_seconds(nuclide: str) -> float:
    """Halveringstid i sekunder (radioactivedecay hvis mulig, ellers fallback)."""
    if RD_AVAILABLE:
        try:
            return float(rd.Nuclide(nuclide).half_life("s"))  # type: ignore[attr-defined]
        except Exception:
            pass
    return float(FALLBACK_T12_S[nuclide])

def human_readable_time(seconds: float) -> str:
    """Gjør sekunder om til en kort lesbar streng (velger mest passende enhet)."""
    units = [
        ("år", 365.25 * 86400),
        ("d", 86400),
        ("t", 3600),
        ("min", 60),
        ("s", 1),
    ]
    s = max(0.0, float(seconds))
    for name, size in units:
        if s >= size or name == "s":
            val = s / size
            # Bruk maks 3 signifikante siffer
            if val >= 100:
                fmt = f"{val:,.0f}"
            elif val >= 10:
                fmt = f"{val:,.1f}"
            else:
                fmt = f"{val:,.2f}"
            # Norsk desimal-komma
            fmt = fmt.replace(",", "X").replace(".", ",").replace("X", ".")
            return f"{fmt} {name}"
    return "0 s"

# ---------------------------------------------------
# GUI-applikasjon
# ---------------------------------------------------
class FysikkmesterApp:
    def __init__(self, root: tk.Tk):
        self.root = root
        self.root.title("Fysikkmester – Henfall og halveringstid")
        self.root.geometry("760x580")

        notebook = ttk.Notebook(self.root)
        notebook.pack(fill="both", expand=True)

        self.tab_henfall = ttk.Frame(notebook)
        self.tab_halver = ttk.Frame(notebook)

        notebook.add(self.tab_henfall, text="Henfall")
        notebook.add(self.tab_halver, text="Halveringstid")

        self._bygg_tab_henfall()
        self._bygg_tab_halver()

    # ---------------------- Henfall ----------------------
    def _bygg_tab_henfall(self):
        ramme = ttk.Frame(self.tab_henfall, padding=16)
        ramme.pack(fill="both", expand=True)

        instr = ("Skriv inn isotopnavn (f.eks. U-238, 238U, C-14). "
                 "Dersom isotopnavn er tomt, brukes A og Z. Velg strålingstype.")
        ttk.Label(ramme, text=instr, wraplength=720, justify="left").grid(row=0, column=0, columnspan=6, sticky="w", pady=(0,10))

        # Isotopnavn (valgfritt)
        ttk.Label(ramme, text="Isotopnavn:").grid(row=1, column=0, sticky="e", padx=(0,8), pady=4)
        self.entry_iso = ttk.Entry(ramme, width=18)
        self.entry_iso.grid(row=1, column=1, sticky="w", pady=4)
        self.entry_iso.insert(0, "C-14")

        ttk.Label(ramme, text="A (nukleontall):").grid(row=1, column=2, sticky="e", padx=(20,8))
        self.entry_A = ttk.Entry(ramme, width=10)
        self.entry_A.grid(row=1, column=3, sticky="w")
        self.entry_A.insert(0, "14")

        ttk.Label(ramme, text="Z (protonstall):").grid(row=1, column=4, sticky="e", padx=(20,8))
        self.entry_Z = ttk.Entry(ramme, width=10)
        self.entry_Z.grid(row=1, column=5, sticky="w")
        self.entry_Z.insert(0, "6")

        ttk.Label(ramme, text="Strålingstype:").grid(row=2, column=0, sticky="e", padx=(0,8), pady=6)
        self.decay_var = tk.StringVar(value="alfa")
        decay_combo = ttk.Combobox(ramme, textvariable=self.decay_var, state="readonly",
                                   values=["alfa (α)","beta- (β⁻)","beta+ (β⁺)","gamma (γ)"], width=16)
        decay_combo.grid(row=2, column=1, sticky="w", pady=6)

        btn = ttk.Button(ramme, text="Beregn henfall", command=self._beregn_henfall)
        btn.grid(row=2, column=5, sticky="e", pady=6)

        ttk.Separator(ramme, orient="horizontal").grid(row=3, column=0, columnspan=6, sticky="ew", pady=12)

        self.result_isotop = tk.StringVar(value="Resultat: –")
        lbl_res = ttk.Label(ramme, textvariable=self.result_isotop, font=("TkDefaultFont", 11, "bold"))
        lbl_res.grid(row=4, column=0, columnspan=6, sticky="w", pady=(0,6))

        ttk.Label(ramme, text="Forklaring:").grid(row=5, column=0, sticky="nw")
        self.txt_forklaring = tk.Text(ramme, height=10, width=92, wrap="word")
        self.txt_forklaring.grid(row=5, column=1, columnspan=5, sticky="nsew")
        ramme.grid_rowconfigure(5, weight=1)
        ramme.grid_columnconfigure(5, weight=1)

        # Enter-taster
        self.entry_iso.bind("<Return>", lambda e: self._beregn_henfall())
        self.entry_A.bind("<Return>", lambda e: self._beregn_henfall())
        self.entry_Z.bind("<Return>", lambda e: self._beregn_henfall())

    def _parse_AZ_fra_gui(self):
        """
        Prøv først isotopnavn (hvis feltet ikke er tomt). Ellers bruk A/Z.
        Returnerer (A, Z).
        """
        iso_txt = (self.entry_iso.get() or "").strip()
        if iso_txt:
            A, Z = parse_isotopnavn(iso_txt)
            return A, Z
        # Fall tilbake til manuell A/Z
        try:
            A = int(self.entry_A.get())
            Z = int(self.entry_Z.get())
            if A <= 0 or Z < 0 or Z >= len(ELEMENT_SYMBOLS) or Z > A:
                raise ValueError
            return A, Z
        except Exception:
            raise ValueError("Kontroller isotopnavn ELLER at A og Z er heltall med Z≤A og gyldige verdier.")

    def _beregn_henfall(self):
        try:
            A, Z = self._parse_AZ_fra_gui()
            dtype = self.decay_var.get()
            if dtype.startswith("alfa"):
                isotop, forklaring = do_alpha(A, Z)
            elif dtype.startswith("beta-"):
                isotop, forklaring = do_beta_minus(A, Z)
            elif dtype.startswith("beta+"):
                isotop, forklaring = do_beta_plus(A, Z)
            else:
                isotop, forklaring = do_gamma(A, Z)

            sym = z_to_symbol(Z)
            N = A - Z
            # Skriv til GUI
            self.result_isotop.set(f"Resultat: {isotop}")
            self.txt_forklaring.config(state="normal")
            self.txt_forklaring.delete("1.0", "end")
            self.txt_forklaring.insert("1.0",
                f"Inndata: {sym}-{A} (Z={Z}, N={N}).\n{forklaring}\n"
            )
            self.txt_forklaring.config(state="disabled")
        except Exception as ex:
            messagebox.showerror("Feil i henfall", str(ex))

    # ------------------- Halveringstid -------------------
    def _bygg_tab_halver(self):
        ramme = ttk.Frame(self.tab_halver, padding=16)
        ramme.pack(fill="both", expand=True)

        instr = ("Velg isotop (fra lista) og oppgi startmasse m₀ (gram).\n"
                 "Velg inndatamodus: *Antall halveringer n* eller *Tid t*. "
                 "Programmet beregner gjenstående masse m = m₀·(1/2)ⁿ og tid t = n·T₁/₂ (eller omvendt).")
        ttk.Label(ramme, text=instr, wraplength=720, justify="left").grid(row=0, column=0, columnspan=6, sticky="w", pady=(0,10))

        # Isotop + T1/2
        ttk.Label(ramme, text="Isotop:").grid(row=1, column=0, sticky="e", padx=(0,8), pady=4)
        self.iso_var = tk.StringVar(value=ISOTOP_LISTE[0])
        iso_combo = ttk.Combobox(ramme, textvariable=self.iso_var, values=ISOTOP_LISTE, state="readonly", width=14)
        iso_combo.grid(row=1, column=1, sticky="w", pady=4)
        iso_combo.bind("<<ComboboxSelected>>", lambda e: self._oppdater_T12())

        ttk.Label(ramme, text="T₁/₂ (halveringstid):").grid(row=1, column=2, sticky="e", padx=(20,8))
        self.lbl_t12 = ttk.Label(ramme, text="–")
        self.lbl_t12.grid(row=1, column=3, sticky="w")

        # Startmasse
        ttk.Label(ramme, text="Startmasse m₀ (g):").grid(row=1, column=4, sticky="e", padx=(20,8))
        self.entry_m0 = ttk.Entry(ramme, width=12)
        self.entry_m0.grid(row=1, column=5, sticky="w", pady=4)
        self.entry_m0.insert(0, "100")

        # Modusvalg
        ttk.Label(ramme, text="Inndatamodus:").grid(row=2, column=0, sticky="e", padx=(0,8), pady=(10,4))
        self.mode_var = tk.StringVar(value="n")
        rb_n = ttk.Radiobutton(ramme, text="Antall halveringer (n)", variable=self.mode_var, value="n",
                               command=self._oppdater_modus)
        rb_t = ttk.Radiobutton(ramme, text="Tid (t)", variable=self.mode_var, value="t",
                               command=self._oppdater_modus)
        rb_n.grid(row=2, column=1, sticky="w", pady=(10,4))
        rb_t.grid(row=2, column=2, sticky="w", pady=(10,4))

        # Inndatafelter for n
        ttk.Label(ramme, text="n:").grid(row=3, column=0, sticky="e", padx=(0,8), pady=4)
        self.entry_n = ttk.Entry(ramme, width=12)
        self.entry_n.grid(row=3, column=1, sticky="w", pady=4)
        self.entry_n.insert(0, "3")

        # Inndatafelter for t (verdi + enhet)
        ttk.Label(ramme, text="t:").grid(row=3, column=2, sticky="e", padx=(20,8), pady=4)
        self.entry_tval = ttk.Entry(ramme, width=12)
        self.entry_tval.grid(row=3, column=3, sticky="w", pady=4)
        self.entry_tval.insert(0, "10")
        ttk.Label(ramme, text="enhet:").grid(row=3, column=4, sticky="e", padx=(20,8))
        self.combo_tunit = ttk.Combobox(ramme, state="readonly", width=8,
                                        values=["s", "min", "t", "d", "år"])
        self.combo_tunit.grid(row=3, column=5, sticky="w", pady=4)
        self.combo_tunit.set("d")

        # Beregn-knapp
        btn = ttk.Button(ramme, text="Beregn", command=self._beregn_halvering)
        btn.grid(row=4, column=5, sticky="e", pady=8)

        ttk.Separator(ramme, orient="horizontal").grid(row=5, column=0, columnspan=6, sticky="ew", pady=12)

        # Resultater
        self.lbl_m_igjen = tk.StringVar(value="Gjenstående masse: –")
        ttk.Label(ramme, textvariable=self.lbl_m_igjen, font=("TkDefaultFont", 11, "bold")).grid(row=6, column=0, columnspan=6, sticky="w", pady=(0,4))

        self.lbl_tid = tk.StringVar(value="Medgått tid: –")
        ttk.Label(ramme, textvariable=self.lbl_tid, font=("TkDefaultFont", 11, "bold")).grid(row=7, column=0, columnspan=6, sticky="w", pady=(0,4))

        self.lbl_n = tk.StringVar(value="Antall halveringer n: –")
        ttk.Label(ramme, textvariable=self.lbl_n, font=("TkDefaultFont", 11, "bold")).grid(row=8, column=0, columnspan=6, sticky="w")

        # Init visning
        self._oppdater_T12()
        self._oppdater_modus()

        # Enter-taster
        self.entry_n.bind("<Return>", lambda e: self._beregn_halvering())
        self.entry_tval.bind("<Return>", lambda e: self._beregn_halvering())
        self.entry_m0.bind("<Return>", lambda e: self._beregn_halvering())

        # Hint om datakilde
        kilde = "radioactivedecay (eksakt)" if RD_AVAILABLE else "innebygd tabell (omtrentlig)"
        ttk.Label(ramme, text=f"Halveringstider fra: {kilde}.", foreground="#555").grid(row=9, column=0, columnspan=6, sticky="w", pady=(10,0))

    def _oppdater_T12(self):
        iso = self.iso_var.get()
        try:
            t12_str = rd_half_life_readable(iso)
        except Exception:
            t12_str = "ukjent"
        self.lbl_t12.config(text=t12_str)

    def _oppdater_modus(self):
        modus = self.mode_var.get()
        # Aktiver/deaktiver relevante felt
        if modus == "n":
            self.entry_n.config(state="normal")
            self.entry_tval.config(state="disabled")
            self.combo_tunit.config(state="disabled")
        else:
            self.entry_n.config(state="disabled")
            self.entry_tval.config(state="normal")
            self.combo_tunit.config(state="readonly")

    def _beregn_halvering(self):
        try:
            iso = self.iso_var.get()
            m0 = float(self.entry_m0.get())
            if m0 < 0:
                raise ValueError("m₀ må være ≥ 0.")
            T12_s = rd_half_life_seconds(iso)

            modus = self.mode_var.get()
            if modus == "n":
                # Inndata: n -> ut: m og t
                n = float(self.entry_n.get())
                if n < 0:
                    raise ValueError("n må være ≥ 0.")
                t_s = n * T12_s
            else:
                # Inndata: t -> ut: n og m
                t_val = float(self.entry_tval.get())
                if t_val < 0:
                    raise ValueError("t må være ≥ 0.")
                enhet = self.combo_tunit.get()
                faktor = {"s": 1, "min": 60, "t": 3600, "d": 86400, "år": 365.25*86400}[enhet]
                t_s = t_val * faktor
                n = t_s / T12_s if T12_s > 0 else 0.0

            # Masse etter n halveringer
            m = m0 * (0.5 ** n)

            # Formater resultater
            m_str = self._format_mass(m)
            m_pct = "0,00" if m0 == 0 else self._fmt_pct(100 * (m / m0))
            self.lbl_m_igjen.set(f"Gjenstående masse: {m_str} ({m_pct} % av start)")
            self.lbl_tid.set(f"Medgått tid: {human_readable_time(t_s)}")
            self.lbl_n.set(f"Antall halveringer n: {self._fmt_smart(n)}")
        except Exception as ex:
            messagebox.showerror("Feil i beregning",
                                 f"Kunne ikke beregne. Detaljer:\n{ex}")

    # ------------------- Format-hjelpere -------------------
    @staticmethod
    def _format_mass(m_g: float) -> str:
        """Enkel format for gram med passende enhet."""
        m = float(m_g)
        if m >= 1_000_000:
            val, unit = (m / 1_000_000), "tonn"
        elif m >= 1000:
            val, unit = (m / 1000), "kg"
        elif m >= 1:
            val, unit = m, "g"
        elif m >= 1e-3:
            val, unit = (m * 1e3), "mg"
        elif m >= 1e-6:
            val, unit = (m * 1e6), "µg"
        else:
            val, unit = (m * 1e9), "ng"
        if val >= 100:
            s = f"{val:,.0f}"
        elif val >= 10:
            s = f"{val:,.1f}"
        else:
            s = f"{val:,.2f}"
        s = s.replace(",", "X").replace(".", ",").replace("X", ".")
        return f"{s} {unit}"

    @staticmethod
    def _fmt_pct(x: float) -> str:
        """Format prosent med norsk komma."""
        if abs(x) >= 100:
            s = f"{x:,.0f}"
        elif abs(x) >= 10:
            s = f"{x:,.1f}"
        else:
            s = f"{x:,.2f}"
        return s.replace(",", "X").replace(".", ",").replace("X", ".")

    @staticmethod
    def _fmt_smart(x: float) -> str:
        """Pen formatering for reelle tall (brukes for n)."""
        ax = abs(x)
        if ax >= 100:
            s = f"{x:,.0f}"
        elif ax >= 10:
            s = f"{x:,.2f}"
        elif ax >= 1:
            s = f"{x:,.3f}"
        elif ax >= 1e-3:
            s = f"{x:,.4f}"
        else:
            s = f"{x:.4e}"
        return s.replace(",", "X").replace(".", ",").replace("X", ".")

# ---------------------------------------------------
# Main
# ---------------------------------------------------
if __name__ == "__main__":
    root = tk.Tk()
    app = FysikkmesterApp(root)
    root.mainloop()

In [3]:
# Versjon 3
# -*- coding: utf-8 -*-
"""
Fysikkmester GUI – interaktivt verktøy for kjernereaksjoner, halveringstid, aktivitet og kjeder.

Funksjoner:
- Fane "Henfall":
    * Isotopnavn (U-238 / 238U / Tc-99m / 99mTc / C-14) ELLER A og Z.
    * Velg strålingstype (alfa/β-/β+/γ) -> nytt isotop + forklaring.
- Fane "Halveringstid":
    * Velg isotop (eller legg til via søk), oppgi m₀, atommasse (g/mol).
    * Tre inndatamoduser:
        1) Antall halveringer (n) -> m og t
        2) Tid (t + enhet)        -> n og m
        3) Sluttmasse (m)         -> n og t
    * Viser også aktivitet: A₀ (start) og A(t) i Bq (SI-prefiks).
    * Graf (Canvas), logaritmisk y-akse (auto/manuell), eksport til PNG (eller PS).
- Fane "Kjede":
    * Automatisk kjede (hvis radioactivedecay finnes): bygg tre av døtre per steg (med grener).
    * Manuell kjede: skriv sekvens (f.eks. "α, β-, α") -> vis isotopene i rekkefølge.

Avhenger kun av standardbibliotek + Tkinter. 'radioactivedecay' og 'Pillow' er valgfritt.
"""

import io
import math
import re
import tkinter as tk
from tkinter import ttk, messagebox, filedialog, simpledialog

# ---------------------------------------------------
# Valgfri integrasjon
# ---------------------------------------------------
RD_AVAILABLE = False
try:
    import radioactivedecay as rd  # type: ignore
    RD_AVAILABLE = True
except Exception:
    RD_AVAILABLE = False

PIL_AVAILABLE = False
try:
    from PIL import Image, EpsImagePlugin  # type: ignore
    PIL_AVAILABLE = True
    # Forsøk å sette Ghostscript-binærnavn (kan være nødvendig på noen systemer)
    EpsImagePlugin.gs_windows_binary = "gswin64c"
    EpsImagePlugin.gs_linux_binary = "gs"
except Exception:
    PIL_AVAILABLE = False

# Fysiske konstanter
N_A = 6.02214076e23  # 1/mol
LN2 = math.log(2.0)

# ---------------------------------------------------
# Periodetabell (Z -> symbol) og omvendt
# ---------------------------------------------------
ELEMENT_SYMBOLS = [
    None,
    "H","He","Li","Be","B","C","N","O","F","Ne",
    "Na","Mg","Al","Si","P","S","Cl","Ar","K","Ca",
    "Sc","Ti","V","Cr","Mn","Fe","Co","Ni","Cu","Zn",
    "Ga","Ge","As","Se","Br","Kr","Rb","Sr","Y","Zr",
    "Nb","Mo","Tc","Ru","Rh","Pd","Ag","Cd","In","Sn",
    "Sb","Te","I","Xe","Cs","Ba","La","Ce","Pr","Nd",
    "Pm","Sm","Eu","Gd","Tb","Dy","Ho","Er","Tm","Yb",
    "Lu","Hf","Ta","W","Re","Os","Ir","Pt","Au","Hg",
    "Tl","Pb","Bi","Po","At","Rn","Fr","Ra","Ac","Th",
    "Pa","U","Np","Pu","Am","Cm","Bk","Cf","Es","Fm",
    "Md","No","Lr","Rf","Db","Sg","Bh","Hs","Mt","Ds",
    "Rg","Cn","Nh","Fl","Mc","Lv","Ts","Og"
]
SYMBOL_TO_Z = {sym: z for z, sym in enumerate(ELEMENT_SYMBOLS) if sym}
SYMBOL_UPPER_TO_CANON = {sym.upper(): sym for sym in SYMBOL_TO_Z.keys()}

# ---------------------------------------------------
# Partikler
# ---------------------------------------------------
class Partikkel:
    def __init__(self, navn, A, Z, symbol):
        self.navn = navn
        self._A = A
        self._Z = Z
        self.symbol = symbol
    def A(self): return self._A
    def Z(self): return self._Z

ALFA    = Partikkel("alfa",    4, +2, "α")
E_M     = Partikkel("elektron (beta-)", 0, -1, "e-")
E_P     = Partikkel("positron (beta+)", 0, +1, "e+")
GAMMA   = Partikkel("gamma",   0,  0, "γ")

# ---------------------------------------------------
# Henfallsregler og hjelpere
# ---------------------------------------------------
def z_to_symbol(Z: int) -> str:
    return ELEMENT_SYMBOLS[Z] if 1 <= Z < len(ELEMENT_SYMBOLS) else "?"

def make_isotope(Z: int, A: int) -> str:
    if A <= 0 or Z < 0 or Z >= len(ELEMENT_SYMBOLS):
        raise ValueError(f"Ugyldig kjerne (A={A}, Z={Z}).")
    sym = z_to_symbol(Z)
    if sym == "?":
        raise ValueError(f"Ukjent grunnstoff for Z={Z}.")
    return f"{sym}-{A}"

def do_alpha(A: int, Z: int):
    if A < 5 or Z < 3:
        raise ValueError("Alfa-henfall krever A≥5 og Z≥3.")
    A2, Z2 = A - 4, Z - 2
    isotop = make_isotope(Z2, A2)
    forklaring = (f"Alfa (α): {z_to_symbol(Z)}-{A} → {isotop} + α\n"
                  f"A: {A}→{A2} (−4), Z: {Z}→{Z2} (−2), N: {A-Z}→{A2-Z2} (−2).")
    return isotop, forklaring

def do_beta_minus(A: int, Z: int):
    A2, Z2 = A, Z + 1
    isotop = make_isotope(Z2, A2)
    forklaring = (f"Beta- (β⁻): {z_to_symbol(Z)}-{A} → {isotop} + e⁻ + ν̄ₑ\n"
                  f"A uendret, Z: {Z}→{Z2} (+1), N: {A-Z}→{A2-Z2} (−1).")
    return isotop, forklaring

def do_beta_plus(A: int, Z: int):
    if Z < 1:
        raise ValueError("Beta+ krever Z≥1.")
    A2, Z2 = A, Z - 1
    isotop = make_isotope(Z2, A2)
    forklaring = (f"Beta+ (β⁺): {z_to_symbol(Z)}-{A} → {isotop} + e⁺ + νₑ\n"
                  f"A uendret, Z: {Z}→{Z2} (−1), N: {A-Z}→{A2-Z2} (+1).\n"
                  f"*Merk:* β⁺ krever energioverskudd eller elektroninnfangning i praksis.")
    return isotop, forklaring

def do_gamma(A: int, Z: int):
    isotop = make_isotope(Z, A)
    forklaring = (f"Gamma (γ): {z_to_symbol(Z)}-{A}* → {isotop} + γ\n"
                  f"A, Z, N uendret (de-eksitasjon).")
    return isotop, forklaring

# ---------------------------------------------------
# Isotop-parsing (brukerformat: "U-238", "238U", "Tc-99m", "99mTc", "C-14" osv.)
# ---------------------------------------------------
ISO_PATTERNS = [
    re.compile(r"^\s*([A-Za-z]{1,3})\s*[-\s]?\s*(\d{1,3})(m?)\s*$"),  # U-238 / U238 / Na-22 / Tc-99m
    re.compile(r"^\s*(\d{1,3})(m?)\s*([A-Za-z]{1,3})\s*$"),           # 238U / 99mTc
]

def normaliser_symbol(sym_inn: str) -> str:
    key = sym_inn.strip().upper()
    if key in SYMBOL_UPPER_TO_CANON:
        return SYMBOL_UPPER_TO_CANON[key]
    raise ValueError(f"Ukjent kjemisk symbol: '{sym_inn}'.")

def parse_isotopnavn(tekst: str):
    """
    Returnerer (label_str, base_symbol, A, is_metastable) fra strenger som:
    'U-238', '238U', 'C-14', 'na-22', 'Tc-99m', '99mTc'.
    """
    t = (tekst or "").strip()
    if not t:
        raise ValueError("Tom isotopstreng.")
    for pat in ISO_PATTERNS:
        m = pat.match(t)
        if m:
            if pat is ISO_PATTERNS[0]:
                sym_raw, A_str, mflag = m.groups()
            else:
                A_str, mflag, sym_raw = m.groups()
            sym = normaliser_symbol(sym_raw)
            Z = SYMBOL_TO_Z[sym]
            A = int(A_str)
            if A <= 0 or Z <= 0 or Z > A:
                raise ValueError(f"Ugyldig isotop: {sym}-{A}.")
            is_m = (mflag == "m")
            label = f"{sym}-{A}{'m' if is_m else ''}"
            return label, sym, A, is_m
    raise ValueError("Ugyldig isotopformat. Eksempler: 'U-238', '238U', 'Tc-99m', '99mTc'.")

# ---------------------------------------------------
# Halveringstid-data (fallback) i sekunder
# ---------------------------------------------------
DAY = 86400.0
YEAR = 365.25 * DAY
HOUR = 3600.0

FALLBACK_T12_S = {
    # Tidligere og utvidet utvalg (omtrentlig):
    "C-14": 5730 * YEAR,
    "U-238": 4.468e9 * YEAR,
    "Co-60": 5.2714 * YEAR,
    "I-131": 8.0197 * DAY,
    "Cs-137": 30.17 * YEAR,
    "Rn-222": 3.8235 * DAY,
    "K-40": 1.251e9 * YEAR,
    "Sr-90": 28.90 * YEAR,
    "Po-210": 138.376 * DAY,
    "Ra-226": 1600 * YEAR,
    "Na-22": 2.6018 * YEAR,
    "Th-232": 1.405e10 * YEAR,
    "H-3": 12.32 * YEAR,
    "Be-7": 53.22 * DAY,
    "Na-24": 14.997 * HOUR,
    "P-32": 14.268 * DAY,
    "S-35": 87.51 * DAY,
    "K-42": 12.36 * HOUR,
    "Ca-45": 162.61 * DAY,
    "Fe-59": 44.495 * DAY,
    "Co-57": 271.8 * DAY,
    "Ni-63": 101.2 * YEAR,
    "Zn-65": 243.9 * DAY,
    "Sr-89": 50.57 * DAY,
    "Y-90": 64.053 * HOUR,
    "Tc-99m": 6.0067 * HOUR,
    "Tc-99": 2.111e5 * YEAR,
    "I-123": 13.22 * HOUR,
    "I-125": 59.49 * DAY,
    "Xe-133": 5.243 * DAY,
    "Ba-133": 10.51 * YEAR,
    "Pb-210": 22.3 * YEAR,
    "Bi-210": 5.012 * DAY,
    "Po-214": 164.3e-6,
    "U-235": 7.04e8 * YEAR,
    "Pu-239": 24110 * YEAR,
    "Am-241": 432.2 * YEAR,
    "C-11": 20.334 * 60.0,
    "N-13": 9.97 * 60.0,
    "O-15": 122.24,
    "F-18": 109.77 * 60.0,
}
ISOTOP_LISTE = sorted(FALLBACK_T12_S.keys())

def rd_half_life_readable(nuclide: str) -> str:
    if RD_AVAILABLE:
        try:
            return rd.Nuclide(nuclide).half_life("readable")  # type: ignore[attr-defined]
        except Exception:
            pass
    secs = FALLBACK_T12_S.get(nuclide)
    if secs is None:
        return "ukjent"
    return human_readable_time(secs)

def rd_half_life_seconds(nuclide: str) -> float:
    if RD_AVAILABLE:
        try:
            return float(rd.Nuclide(nuclide).half_life("s"))  # type: ignore[attr-defined]
        except Exception:
            pass
    if nuclide in FALLBACK_T12_S:
        return float(FALLBACK_T12_S[nuclide])
    raise ValueError(f"Mangler halveringstid for '{nuclide}'.")

def human_readable_time(seconds: float) -> str:
    units = [("år", 365.25*86400), ("d", 86400), ("t", 3600), ("min", 60), ("s", 1)]
    s = max(0.0, float(seconds))
    for name, size in units:
        if s >= size or name == "s":
            val = s / size
            if val >= 100: fmt = f"{val:,.0f}"
            elif val >= 10: fmt = f"{val:,.1f}"
            else: fmt = f"{val:,.2f}"
            fmt = fmt.replace(",", "X").replace(".", ",").replace("X", ".")
            return f"{fmt} {name}"
    return "0 s"

# ---------------------------------------------------
# GUI-applikasjon
# ---------------------------------------------------
class FysikkmesterApp:
    def __init__(self, root: tk.Tk):
        self.root = root
        self.root.title("Fysikkmester – Henfall, halveringstid, aktivitet og kjeder")
        self.root.geometry("1080x900")

        notebook = ttk.Notebook(self.root)
        notebook.pack(fill="both", expand=True)

        self.tab_henfall = ttk.Frame(notebook)
        self.tab_halver = ttk.Frame(notebook)
        self.tab_kjede  = ttk.Frame(notebook)

        notebook.add(self.tab_henfall, text="Henfall")
        notebook.add(self.tab_halver, text="Halveringstid")
        notebook.add(self.tab_kjede,  text="Kjede")

        self._bygg_tab_henfall()
        self._bygg_tab_halver()
        self._bygg_tab_kjede()

    # ---------------------- Henfall ----------------------
    def _bygg_tab_henfall(self):
        ramme = ttk.Frame(self.tab_henfall, padding=16)
        ramme.pack(fill="both", expand=True)

        instr = ("Skriv inn isotopnavn (U-238, 238U, Tc-99m, 99mTc, C-14). "
                 "Hvis tomt, brukes A og Z. Velg strålingstype.")
        ttk.Label(ramme, text=instr, wraplength=1040, justify="left").grid(row=0, column=0, columnspan=6, sticky="w", pady=(0,10))

        ttk.Label(ramme, text="Isotopnavn:").grid(row=1, column=0, sticky="e", padx=(0,8), pady=4)
        self.entry_iso = ttk.Entry(ramme, width=18)
        self.entry_iso.grid(row=1, column=1, sticky="w", pady=4)
        self.entry_iso.insert(0, "C-14")

        ttk.Label(ramme, text="A (nukleontall):").grid(row=1, column=2, sticky="e", padx=(20,8))
        self.entry_A = ttk.Entry(ramme, width=10)
        self.entry_A.grid(row=1, column=3, sticky="w")
        self.entry_A.insert(0, "14")

        ttk.Label(ramme, text="Z (protonstall):").grid(row=1, column=4, sticky="e", padx=(20,8))
        self.entry_Z = ttk.Entry(ramme, width=10)
        self.entry_Z.grid(row=1, column=5, sticky="w")
        self.entry_Z.insert(0, "6")

        ttk.Label(ramme, text="Strålingstype:").grid(row=2, column=0, sticky="e", padx=(0,8), pady=6)
        self.decay_var = tk.StringVar(value="alfa")
        decay_combo = ttk.Combobox(ramme, textvariable=self.decay_var, state="readonly",
                                   values=["alfa (α)","beta- (β⁻)","beta+ (β⁺)","gamma (γ)"], width=16)
        decay_combo.grid(row=2, column=1, sticky="w", pady=6)

        btn = ttk.Button(ramme, text="Beregn henfall", command=self._beregn_henfall)
        btn.grid(row=2, column=5, sticky="e", pady=6)

        ttk.Separator(ramme, orient="horizontal").grid(row=3, column=0, columnspan=6, sticky="ew", pady=12)

        self.result_isotop = tk.StringVar(value="Resultat: –")
        ttk.Label(ramme, textvariable=self.result_isotop, font=("TkDefaultFont", 11, "bold")).grid(row=4, column=0, columnspan=6, sticky="w", pady=(0,6))

        ttk.Label(ramme, text="Forklaring:").grid(row=5, column=0, sticky="nw")
        self.txt_forklaring = tk.Text(ramme, height=12, width=120, wrap="word")
        self.txt_forklaring.grid(row=5, column=1, columnspan=5, sticky="nsew")
        ramme.grid_rowconfigure(5, weight=1)
        ramme.grid_columnconfigure(5, weight=1)

        self.entry_iso.bind("<Return>", lambda e: self._beregn_henfall())
        self.entry_A.bind("<Return>", lambda e: self._beregn_henfall())
        self.entry_Z.bind("<Return>", lambda e: self._beregn_henfall())

    def _parse_AZ_fra_gui(self):
        iso_txt = (self.entry_iso.get() or "").strip()
        if iso_txt:
            label, sym, A, is_m = parse_isotopnavn(iso_txt)
            Z = SYMBOL_TO_Z[sym]
            return A, Z
        try:
            A = int(self.entry_A.get()); Z = int(self.entry_Z.get())
            if A <= 0 or Z < 0 or Z >= len(ELEMENT_SYMBOLS) or Z > A:
                raise ValueError
            return A, Z
        except Exception:
            raise ValueError("Kontroller isotopnavn ELLER at A og Z er heltall med Z≤A og gyldige verdier.")

    def _beregn_henfall(self):
        try:
            A, Z = self._parse_AZ_fra_gui()
            dtype = self.decay_var.get()
            if dtype.startswith("alfa"): isotop, forklaring = do_alpha(A, Z)
            elif dtype.startswith("beta-"): isotop, forklaring = do_beta_minus(A, Z)
            elif dtype.startswith("beta+"): isotop, forklaring = do_beta_plus(A, Z)
            else: isotop, forklaring = do_gamma(A, Z)
            sym = z_to_symbol(Z); N = A - Z
            self.result_isotop.set(f"Resultat: {isotop}")
            self.txt_forklaring.config(state="normal")
            self.txt_forklaring.delete("1.0", "end")
            self.txt_forklaring.insert("1.0", f"Inndata: {sym}-{A} (Z={Z}, N={N}).\n{forklaring}\n")
            self.txt_forklaring.config(state="disabled")
        except Exception as ex:
            messagebox.showerror("Feil i henfall", str(ex))

    # ------------------- Halveringstid -------------------
    def _bygg_tab_halver(self):
        ramme = ttk.Frame(self.tab_halver, padding=16)
        ramme.pack(fill="both", expand=True)

        instr = ("Velg isotop fra lista – eller søk/legg til en isotop. Oppgi startmasse m₀ og atommasse (g/mol).\n"
                 "Velg inndatamodus: *n*, *t* eller *m*. Programmet beregner resten, viser aktivitet, og tegner graf.")
        ttk.Label(ramme, text=instr, wraplength=1040, justify="left").grid(row=0, column=0, columnspan=12, sticky="w", pady=(0,10))

        # Isotop + T1/2
        ttk.Label(ramme, text="Isotop:").grid(row=1, column=0, sticky="e", padx=(0,8), pady=4)
        self.iso_var = tk.StringVar(value=ISOTOP_LISTE[0])
        self.iso_combo = ttk.Combobox(ramme, textvariable=self.iso_var, values=ISOTOP_LISTE, state="readonly", width=16)
        self.iso_combo.grid(row=1, column=1, sticky="w", pady=4)
        self.iso_combo.bind("<<ComboboxSelected>>", lambda e: (self._oppdater_T12(), self._autofyll_atommasse(), self._tegn_graf()))
        self._isotopliste = list(ISOTOP_LISTE)

        ttk.Label(ramme, text="T₁/₂:").grid(row=1, column=2, sticky="e", padx=(20,8))
        self.lbl_t12 = ttk.Label(ramme, text="–")
        self.lbl_t12.grid(row=1, column=3, sticky="w")

        # Startmasse + atommasse
        ttk.Label(ramme, text="Startmasse m₀ (g):").grid(row=1, column=4, sticky="e", padx=(20,8))
        self.entry_m0 = ttk.Entry(ramme, width=12)
        self.entry_m0.grid(row=1, column=5, sticky="w", pady=4)
        self.entry_m0.insert(0, "100")
        self.entry_m0.bind("<KeyRelease>", lambda e: self._tegn_graf())

        ttk.Label(ramme, text="Atommasse (g/mol):").grid(row=1, column=6, sticky="e", padx=(20,8))
        self.entry_M = ttk.Entry(ramme, width=12)
        self.entry_M.grid(row=1, column=7, sticky="w", pady=4)
        self.entry_M.insert(0, "14")  # standard ~ masse-tall
        self.entry_M.bind("<KeyRelease>", lambda e: self._tegn_graf())

        # Søk/legg til isotop
        ttk.Label(ramme, text="Søk/legg til isotop:").grid(row=2, column=0, sticky="e", padx=(0,8), pady=4)
        self.entry_search_iso = ttk.Entry(ramme, width=16)
        self.entry_search_iso.grid(row=2, column=1, sticky="w", pady=4)
        self.entry_search_iso.insert(0, "U-235")
        btn_add = ttk.Button(ramme, text="Legg til →", command=self._legg_til_isotop)
        btn_add.grid(row=2, column=2, sticky="w", pady=4)
        self.entry_search_iso.bind("<Return>", lambda e: self._legg_til_isotop())

        # Modusvalg
        ttk.Label(ramme, text="Inndatamodus:").grid(row=3, column=0, sticky="e", padx=(0,8), pady=(10,4))
        self.mode_var = tk.StringVar(value="n")
        rb_n = ttk.Radiobutton(ramme, text="n (halveringer)", variable=self.mode_var, value="n",
                               command=lambda: (self._oppdater_modus(), self._tegn_graf()))
        rb_t = ttk.Radiobutton(ramme, text="t (tid)", variable=self.mode_var, value="t",
                               command=lambda: (self._oppdater_modus(), self._tegn_graf()))
        rb_m = ttk.Radiobutton(ramme, text="m (sluttmasse)", variable=self.mode_var, value="m",
                               command=lambda: (self._oppdater_modus(), self._tegn_graf()))
        rb_n.grid(row=3, column=1, sticky="w", pady=(10,4))
        rb_t.grid(row=3, column=2, sticky="w", pady=(10,4))
        rb_m.grid(row=3, column=3, sticky="w", pady=(10,4))

        # Inndata-felter
        ttk.Label(ramme, text="n:").grid(row=4, column=0, sticky="e", padx=(0,8), pady=4)
        self.entry_n = ttk.Entry(ramme, width=12); self.entry_n.grid(row=4, column=1, sticky="w", pady=4); self.entry_n.insert(0, "3")

        ttk.Label(ramme, text="t:").grid(row=4, column=2, sticky="e", padx=(20,8), pady=4)
        self.entry_tval = ttk.Entry(ramme, width=12); self.entry_tval.grid(row=4, column=3, sticky="w", pady=4); self.entry_tval.insert(0, "10")
        self.entry_tval.bind("<KeyRelease>", lambda e: self._tegn_graf())
        ttk.Label(ramme, text="enhet:").grid(row=4, column=4, sticky="e", padx=(20,8))
        self.combo_tunit = ttk.Combobox(ramme, state="readonly", width=8, values=["s", "min", "t", "d", "år"])
        self.combo_tunit.grid(row=4, column=5, sticky="w", pady=4); self.combo_tunit.set("d")
        self.combo_tunit.bind("<<ComboboxSelected>>", lambda e: self._tegn_graf())

        ttk.Label(ramme, text="Sluttmasse m (g):").grid(row=4, column=6, sticky="e", padx=(20,8), pady=4)
        self.entry_m = ttk.Entry(ramme, width=12); self.entry_m.grid(row=4, column=7, sticky="w", pady=4); self.entry_m.insert(0, "12,5")
        self.entry_m.bind("<KeyRelease>", lambda e: self._tegn_graf())

        # Log-aksevalg
        self.var_log = tk.BooleanVar(value=False)
        self.var_auto_log = tk.BooleanVar(value=True)
        chk_log = ttk.Checkbutton(ramme, text="Logaritmisk y-akse", variable=self.var_log, command=self._tegn_graf)
        chk_auto = ttk.Checkbutton(ramme, text="Auto-log ved stort spenn", variable=self.var_auto_log, command=self._tegn_graf)
        chk_log.grid(row=5, column=1, sticky="w", pady=(6,2))
        chk_auto.grid(row=5, column=2, sticky="w", pady=(6,2))

        # Beregn / Eksport
        btn_calc = ttk.Button(ramme, text="Beregn", command=self._beregn_halvering); btn_calc.grid(row=5, column=7, sticky="e", pady=6)
        btn_export = ttk.Button(ramme, text="Eksporter graf til PNG …", command=self._eksporter_png); btn_export.grid(row=5, column=6, sticky="e", pady=6, padx=(10,0))

        ttk.Separator(ramme, orient="horizontal").grid(row=6, column=0, columnspan=12, sticky="ew", pady=12)

        # Resultater (masse, tid, n, aktivitet)
        self.lbl_m_igjen = tk.StringVar(value="Gjenstående masse: –")
        ttk.Label(ramme, textvariable=self.lbl_m_igjen, font=("TkDefaultFont", 11, "bold")).grid(row=7, column=0, columnspan=12, sticky="w", pady=(0,4))

        self.lbl_tid = tk.StringVar(value="Medgått tid: –")
        ttk.Label(ramme, textvariable=self.lbl_tid, font=("TkDefaultFont", 11, "bold")).grid(row=8, column=0, columnspan=12, sticky="w", pady=(0,4))

        self.lbl_n = tk.StringVar(value="Antall halveringer n: –")
        ttk.Label(ramme, textvariable=self.lbl_n, font=("TkDefaultFont", 11, "bold")).grid(row=9, column=0, columnspan=12, sticky="w", pady=(0,4))

        self.lbl_A0 = tk.StringVar(value="Startaktivitet A₀: –")
        self.lbl_At = tk.StringVar(value="Aktivitet nå A(t): –")
        ttk.Label(ramme, textvariable=self.lbl_A0, font=("TkDefaultFont", 11, "bold")).grid(row=10, column=0, columnspan=12, sticky="w", pady=(4,0))
        ttk.Label(ramme, textvariable=self.lbl_At, font=("TkDefaultFont", 11, "bold")).grid(row=11, column=0, columnspan=12, sticky="w", pady=(0,4))

        # Canvas
        ttk.Separator(ramme, orient="horizontal").grid(row=12, column=0, columnspan=12, sticky="ew", pady=12)
        self.canvas = tk.Canvas(ramme, width=1020, height=360, bg="white", highlightthickness=1, highlightbackground="#ccc")
        self.canvas.grid(row=13, column=0, columnspan=12, sticky="nsew")
        ramme.grid_rowconfigure(13, weight=1)
        for c in range(12):
            ramme.grid_columnconfigure(c, weight=1)

        # Init
        self._oppdater_T12()
        self._autofyll_atommasse()
        self._oppdater_modus()
        self._tegn_graf()

        # Kildeinfo
        kilde = "radioactivedecay (eksakt når tilgjengelig)" if RD_AVAILABLE else "innebygd tabell (omtrentlig)"
        ttk.Label(ramme, text=f"Halveringstider fra: {kilde}.", foreground="#555").grid(row=14, column=0, columnspan=12, sticky="w", pady=(10,0))

        # Enter-taster
        self.entry_n.bind("<Return>", lambda e: self._beregn_halvering())
        self.entry_tval.bind("<Return>", lambda e: self._beregn_halvering())
        self.entry_m0.bind("<Return>", lambda e: self._beregn_halvering())
        self.entry_m.bind("<Return>", lambda e: self._beregn_halvering())
        self.entry_M.bind("<Return>", lambda e: self._beregn_halvering())

    # ------------------- Kjede -------------------
    def _bygg_tab_kjede(self):
        ramme = ttk.Frame(self.tab_kjede, padding=16)
        ramme.pack(fill="both", expand=True)

        ttk.Label(ramme, text="Kjedereaksjons-visning (henfallssekvens)", font=("TkDefaultFont", 12, "bold")).grid(row=0, column=0, columnspan=6, sticky="w", pady=(0,6))

        # Automatisk (med RD)
        frm_auto = ttk.Labelframe(ramme, text="Automatisk kjede (krever radioactivedecay)", padding=10)
        frm_auto.grid(row=1, column=0, columnspan=6, sticky="ew", padx=(0,0), pady=(0,10))

        ttk.Label(frm_auto, text="Startisotop:").grid(row=0, column=0, sticky="e", padx=(0,8))
        self.entry_kjede_start = ttk.Entry(frm_auto, width=18); self.entry_kjede_start.grid(row=0, column=1, sticky="w")
        self.entry_kjede_start.insert(0, "U-238")

        ttk.Label(frm_auto, text="Maks steg:").grid(row=0, column=2, sticky="e", padx=(20,8))
        self.entry_kjede_depth = ttk.Entry(frm_auto, width=8); self.entry_kjede_depth.grid(row=0, column=3, sticky="w")
        self.entry_kjede_depth.insert(0, "6")

        btn_build_auto = ttk.Button(frm_auto, text="Bygg kjede", command=self._bygg_kjede_auto)
        btn_build_auto.grid(row=0, column=5, sticky="e")

        self.txt_kjede_auto = tk.Text(frm_auto, height=14, width=130, wrap="word", state="disabled")
        self.txt_kjede_auto.grid(row=1, column=0, columnspan=6, sticky="nsew", pady=(8,0))
        frm_auto.grid_columnconfigure(5, weight=1)
        frm_auto.grid_rowconfigure(1, weight=1)

        if not RD_AVAILABLE:
            self.txt_kjede_auto.config(state="normal")
            self.txt_kjede_auto.insert("1.0", "radioactivedecay er ikke installert. Installer pakken for automatisk kjede.\n\npip install radioactivedecay")
            self.txt_kjede_auto.config(state="disabled")

        # Manuell
        frm_man = ttk.Labelframe(ramme, text="Manuell kjede (skriv sekvens av henfall)", padding=10)
        frm_man.grid(row=2, column=0, columnspan=6, sticky="ew", pady=(6,0))

        ttk.Label(frm_man, text="Startisotop:").grid(row=0, column=0, sticky="e", padx=(0,8))
        self.entry_kjede_start_manual = ttk.Entry(frm_man, width=18); self.entry_kjede_start_manual.grid(row=0, column=1, sticky="w")
        self.entry_kjede_start_manual.insert(0, "U-238")

        ttk.Label(frm_man, text="Sekvens (f.eks. 'α, α, β-', eller 'alpha,beta-'):").grid(row=0, column=2, sticky="e", padx=(10,8))
        self.entry_kjede_seq = ttk.Entry(frm_man, width=40); self.entry_kjede_seq.grid(row=0, column=3, sticky="w")
        self.entry_kjede_seq.insert(0, "α, α, β-")

        btn_build_man = ttk.Button(frm_man, text="Bygg manuelt", command=self._bygg_kjede_manuell)
        btn_build_man.grid(row=0, column=5, sticky="e")

        self.txt_kjede_man = tk.Text(frm_man, height=10, width=130, wrap="word", state="disabled")
        self.txt_kjede_man.grid(row=1, column=0, columnspan=6, sticky="nsew", pady=(8,0))
        frm_man.grid_columnconfigure(5, weight=1)
        frm_man.grid_rowconfigure(1, weight=1)

    # --------- Isotop-søk/legg til ---------
    def _legg_til_isotop(self):
        try:
            raw = (self.entry_search_iso.get() or "").strip()
            if not raw: return
            label, sym, A, is_m = parse_isotopnavn(raw)
            if RD_AVAILABLE:
                try:
                    _ = float(rd.Nuclide(label).half_life("s"))  # type: ignore[attr-defined]
                except Exception as ex:
                    raise ValueError(f"Fant ikke halveringstid via radioactivedecay for '{label}'. ({ex})")
            else:
                if label not in FALLBACK_T12_S:
                    t_str = simpledialog.askstring("Ny isotop",
                                                   f"Oppgi T₁/₂ for {label}\n"
                                                   f"(tall + enhet: s/min/t/d/år, f.eks. '6 t' eller '5730 år'):")
                    if not t_str: return
                    secs = self._parse_t12_input_to_seconds(t_str)
                    FALLBACK_T12_S[label] = secs
            if label not in self._isotopliste:
                self._isotopliste.append(label); self._isotopliste.sort()
                self.iso_combo.config(values=self._isotopliste)
            self.iso_var.set(label)
            self._oppdater_T12()
            self._autofyll_atommasse()
            self._tegn_graf()
        except Exception as ex:
            messagebox.showerror("Kunne ikke legge til isotop", str(ex))

    def _parse_t12_input_to_seconds(self, text: str) -> float:
        if not text: raise ValueError("Tom halveringstidsverdi.")
        t = text.strip().replace(",", ".")
        m = re.match(r"^\s*([0-9]+(?:\.[0-9]+)?)\s*([A-Za-zæøåÆØÅ]+)?\s*$", t)
        if not m: raise ValueError("Bruk format: tall + enhet (s, min, t, d, år).")
        val = float(m.group(1)); enhet = (m.group(2) or "s").lower()
        enhet = enhet.replace("h", "t")
        faktor_map = {"s":1.0,"sek":1.0,"sekund":1.0,"sekunder":1.0,
                      "min":60.0,"m":60.0,
                      "t":3600.0,"time":3600.0,"timer":3600.0,
                      "d":86400.0,"dag":86400.0,"dager":86400.0,
                      "år":365.25*86400.0,"ar":365.25*86400.0}
        if enhet not in faktor_map: raise ValueError("Ukjent enhet. Bruk s/min/t/d/år.")
        return val * faktor_map[enhet]

    # --------- Halveringstid-hjelpere (UI) ---------
    def _oppdater_T12(self):
        iso = self.iso_var.get()
        try: t12_str = rd_half_life_readable(iso)
        except Exception: t12_str = "ukjent"
        self.lbl_t12.config(text=t12_str)

    def _autofyll_atommasse(self):
        """Sett atommasse ~ masse-tall fra isotoplabel (kan overstyres av bruker)."""
        try:
            label, sym, A, is_m = parse_isotopnavn(self.iso_var.get())
            self.entry_M.delete(0, "end"); self.entry_M.insert(0, str(A))
        except Exception:
            pass

    def _oppdater_modus(self):
        modus = self.mode_var.get()
        self.entry_n.config(state="normal" if modus=="n" else "disabled")
        self.entry_tval.config(state="normal" if modus=="t" else "disabled")
        self.combo_tunit.config(state="readonly" if modus=="t" else "disabled")
        self.entry_m.config(state="normal" if modus=="m" else "disabled")

    def _les_float(self, widget: tk.Entry) -> float:
        txt = (widget.get() or "").strip().replace(",", ".")
        return float(txt)

    def _beregn_halvering(self):
        try:
            iso = self.iso_var.get()
            m0 = self._les_float(self.entry_m0)
            M = self._les_float(self.entry_M)  # g/mol
            if m0 < 0: raise ValueError("m₀ må være ≥ 0.")
            if M <= 0: raise ValueError("Atommasse må være > 0.")
            T12_s = rd_half_life_seconds(iso)
            lam = LN2 / T12_s

            modus = self.mode_var.get()
            if modus == "n":
                n = float(self._les_float(self.entry_n)); 
                if n < 0: raise ValueError("n må være ≥ 0.")
                t_s = n * T12_s
                m = m0 * (0.5 ** n)
            elif modus == "t":
                t_val = self._les_float(self.entry_tval); 
                if t_val < 0: raise ValueError("t må være ≥ 0.")
                faktor = {"s":1,"min":60,"t":3600,"d":86400,"år":365.25*86400}[self.combo_tunit.get()]
                t_s = t_val * faktor
                n = t_s / T12_s if T12_s > 0 else 0.0
                m = m0 * (0.5 ** n)
            else:  # m
                m = self._les_float(self.entry_m)
                if m <= 0: raise ValueError("Sluttmasse m må være > 0.")
                if m0 <= 0: raise ValueError("m₀ må være > 0 for å beregne fra m.")
                if m > m0: raise ValueError("m kan ikke være større enn m₀.")
                n = math.log(m / m0, 0.5)  # = log2(m0/m)
                t_s = n * T12_s

            # Aktivitet: A = λ N = λ * (m/M) * N_A
            A0 = lam * (m0 / M) * N_A
            At = lam * (m  / M) * N_A

            # Vis resultater
            m_str = self._format_mass(m); m_pct = "0,00" if m0 == 0 else self._fmt_pct(100 * (m / m0))
            self.lbl_m_igjen.set(f"Gjenstående masse: {m_str} ({m_pct} % av start)")
            self.lbl_tid.set(f"Medgått tid: {human_readable_time(t_s)}")
            self.lbl_n.set(f"Antall halveringer n: {self._fmt_smart(n)}")
            self.lbl_A0.set(f"Startaktivitet A₀: {self._format_bq(A0)}")
            self.lbl_At.set(f"Aktivitet nå A(t): {self._format_bq(At)}")

            self._tegn_graf(t_mark=t_s, m_mark=m)
        except Exception as ex:
            messagebox.showerror("Feil i beregning", f"Kunne ikke beregne. Detaljer:\n{ex}")

    # ------------------- Graf (Tkinter Canvas) -------------------
    def _tegn_graf(self, t_mark: float | None = None, m_mark: float | None = None):
        try:
            iso = self.iso_var.get()
            T12_s = rd_half_life_seconds(iso)
            m0 = self._try_les_m0()
        except Exception:
            T12_s = None; m0 = None

        W = int(self.canvas.winfo_width() or 1020)
        H = int(self.canvas.winfo_height() or 360)
        PAD_L, PAD_R, PAD_T, PAD_B = 80, 20, 20, 56
        x0, y0 = PAD_L, H - PAD_B
        x1, y1 = W - PAD_R, PAD_T
        plot_w = max(10, x1 - x0); plot_h = max(10, y0 - y1)

        self.canvas.delete("all")
        self.canvas.create_rectangle(0, 0, W, H, fill="white", outline="")

        if T12_s is None or T12_s <= 0 or m0 is None or m0 <= 0:
            self._tegn_akser(x0, y0, x1, y1)
            self.canvas.create_text(W//2, H//2, text="Oppgi gyldig m₀ og isotop for å tegne graf.",
                                    fill="#777", font=("TkDefaultFont", 10, "italic"))
            return

        n_default = 10
        tmax = n_default * T12_s
        if t_mark is not None: tmax = max(tmax, 1.05 * t_mark)
        tmax = max(tmax, T12_s)

        # Auto-log
        use_log = self.var_log.get()
        if self.var_auto_log.get():
            m_at_tmax = m0 * (0.5 ** (tmax / T12_s))
            if (m0 / max(m_at_tmax, 1e-300)) >= 1e3:
                use_log = True

        def X(t): return x0 + (t / tmax) * plot_w
        def Y_lin(m): m = max(0.0, min(m, m0)); return y0 - (m / m0) * plot_h

        m_min_plot = m0 * (0.5 ** (tmax / T12_s))
        m_min_plot = max(m_min_plot, m0 * 1e-9)
        log_den = math.log(m0) - math.log(m_min_plot) if m_min_plot > 0 else 1.0
        def Y_log(m):
            m = max(m, m_min_plot)
            val = (math.log(m) - math.log(m_min_plot)) / max(1e-12, log_den)
            return y0 - val * plot_h

        Y = Y_log if use_log else Y_lin

        # Akser + rutenett
        self._tegn_akser(x0, y0, x1, y1)

        # X-ticks ved k*T1/2
        k_max = int(tmax // T12_s)
        for k in range(k_max + 1):
            tx = k * T12_s; xp = X(tx)
            self.canvas.create_line(xp, y0, xp, y1, fill="#eee")
            lbl = "0" if k == 0 else f"{k}·T₁/₂"
            self.canvas.create_text(xp, y0 + 16, text=lbl, fill="#555", anchor="n")

        # Y-ticks
        if use_log:
            log10_min = math.log10(m_min_plot); log10_max = math.log10(m0)
            dmin = int(math.floor(log10_min)); dmax = int(math.ceil(log10_max))
            for d in range(dmin, dmax + 1):
                m_tick = 10 ** d
                if m_tick < m_min_plot * 0.999 or m_tick > m0 * 1.001: continue
                yp = Y(m_tick)
                self.canvas.create_line(x0, yp, x1, yp, fill="#f1f1f1")
                self.canvas.create_text(x0 - 6, yp, text=self._format_mass(m_tick), fill="#555", anchor="e")
            y_label = "Masse (g, log)"
        else:
            for frac in [0.0, 0.25, 0.5, 0.75, 1.0]:
                m = m0 * frac; yp = Y(m)
                self.canvas.create_line(x0, yp, x1, yp, fill="#f4f4f4")
                self.canvas.create_text(x0 - 6, yp, text=self._format_mass(m), fill="#555", anchor="e")
            y_label = "Masse (g)"

        # Kurve
        pts = []; N = 600
        for i in range(N + 1):
            t = (i / N) * tmax; m = m0 * (0.5 ** (t / T12_s))
            pts.extend([X(t), Y(m)])
        self.canvas.create_line(pts, fill="#1f77b4", width=2, smooth=True)

        # Markør
        if t_mark is not None and m_mark is not None:
            xm, ym = X(min(t_mark, tmax)), Y(m_mark)
            self.canvas.create_line(xm, y0, xm, y1, fill="#c44", dash=(4, 3))
            self.canvas.create_line(x0, ym, x1, ym, fill="#c44", dash=(4, 3))
            r = 4
            self.canvas.create_oval(xm - r, ym - r, xm + r, ym + r, fill="#d62728", outline="")
            self.canvas.create_text(xm + 6, ym - 12, text=f"t={human_readable_time(t_mark)}, m={self._format_mass(m_mark)}",
                                    anchor="w", fill="#c00", font=("TkDefaultFont", 9, "bold"))

        self.canvas.create_text((x0 + x1)//2, y0 + 28, text="Tid", fill="#333")
        self.canvas.create_text(x0 - 48, (y0 + y1)//2, text=y_label, angle=90, fill="#333")

    def _tegn_akser(self, x0, y0, x1, y1):
        self.canvas.create_line(x0, y0, x1, y0, fill="#333", width=1)
        self.canvas.create_line(x0, y0, x0, y1, fill="#333", width=1)
        self.canvas.create_rectangle(x0, y1, x1, y0, outline="#ddd")

    def _try_les_m0(self):
        try: return self._les_float(self.entry_m0)
        except Exception: return None

    # ------------------- Eksport -------------------
    def _eksporter_png(self):
        path = filedialog.asksaveasfilename(title="Lagre graf som PNG", defaultextension=".png",
                                            filetypes=[("PNG-bilde","*.png"),("Alle filer","*.*")])
        if not path: return
        try:
            ps = self.canvas.postscript(colormode='color')
            if PIL_AVAILABLE:
                try:
                    import io
                    bio = io.BytesIO(ps.encode('utf-8'))
                    img = Image.open(bio); img.load(); img.save(path, "PNG")
                    messagebox.showinfo("Eksport fullført", f"Lagret PNG:\n{path}"); return
                except Exception as ex:
                    ps_path = path.rsplit(".", 1)[0] + ".ps"
                    with open(ps_path, "w", encoding="utf-8") as f: f.write(ps)
                    messagebox.showwarning("Eksport delvis",
                        "PS → PNG krever Ghostscript. PostScript lagret som:\n"
                        f"{ps_path}\n\nDetaljer: {ex}")
            else:
                ps_path = path.rsplit(".", 1)[0] + ".ps"
                with open(ps_path, "w", encoding="utf-8") as f: f.write(ps)
                messagebox.showwarning("Pillow mangler",
                    "PNG-eksport krever Pillow + Ghostscript. PS lagret som:\n" + ps_path)
        except Exception as ex:
            messagebox.showerror("Eksport feilet", f"Kunne ikke eksportere grafen.\nDetaljer:\n{ex}")

    # ------------------- Kjede: automatisk (RD) -------------------
    def _bygg_kjede_auto(self):
        self.txt_kjede_auto.config(state="normal")
        self.txt_kjede_auto.delete("1.0", "end")
        try:
            if not RD_AVAILABLE:
                raise RuntimeError("radioactivedecay ikke tilgjengelig.")
            start_raw = (self.entry_kjede_start.get() or "").strip()
            label, sym, A, is_m = parse_isotopnavn(start_raw)
            depth = int(self.entry_kjede_depth.get())
            if depth < 1: depth = 1
            tekst = self._rd_bygg_kjedetekst(label, depth)
            self.txt_kjede_auto.insert("1.0", tekst)
        except Exception as ex:
            self.txt_kjede_auto.insert("1.0", f"Kunne ikke bygge kjede automatisk.\nDetaljer: {ex}")
        finally:
            self.txt_kjede_auto.config(state="disabled")

    def _rd_bygg_kjedetekst(self, start_label: str, depth: int) -> str:
        """
        Best-effort: henter tilgjengelige grener for hvert steg med radioactivedecay.
        Forsøker flere mulige API-attributter; faller pent tilbake ved ukjent API.
        """
        lines = []
        lines.append(f"Start: {start_label}\n")
        frontier = [(start_label, 1.0, "start")]  # (label, samlet BF, beskrivelse)
        for step in range(1, depth+1):
            new_frontier = []
            lines.append(f"Steg {step}:")
            for (lab, bf, _) in frontier:
                try:
                    nu = rd.Nuclide(lab)  # type: ignore[attr-defined]
                    # Prøv å finne decays-lista med (datter, modus, branching)
                    candidates = []
                    # 1) Objektliste på .decays eller .decay_modes
                    for attr in ["decays", "decay_modes", "branches", "modes"]:
                        if hasattr(nu, attr):
                            data = getattr(nu, attr)
                            if isinstance(data, (list, tuple)) and len(data) > 0:
                                candidates = data; break
                    # 2) Hvis ingen, prøv .branching_fractions og .daughters (parallell-lister)
                    if not candidates:
                        dtrs = getattr(nu, "daughters", None)
                        bfs  = getattr(nu, "branching_fractions", None)
                        modes= getattr(nu, "decay_modes", None) if dtrs is None else None
                        if dtrs is not None and bfs is not None:
                            candidates = list(zip(dtrs, bfs, (["?"]*len(dtrs))))
                    if not candidates:
                        lines.append(f"  {lab}: (ingen data/ukjent API)")
                        continue

                    # Normaliser til liste av dict {daughter, mode, bf}
                    norm = []
                    for item in candidates:
                        d = None; mode = "?"
                        br = None
                        # item kan være objekt eller tuple
                        try:
                            # Objekt med attributter
                            d = getattr(item, "daughter", None) or getattr(item, "nuclide", None)
                            mode = getattr(item, "mode", mode)
                            br = getattr(item, "branching_ratio", None) or getattr(item, "branching_fraction", None)
                        except Exception:
                            pass
                        if d is None:
                            # tuple- eller listformat
                            if isinstance(item, (tuple, list)):
                                if len(item) >= 2:
                                    d = str(item[0]); br = float(item[1]) if item[1] is not None else None
                                    if len(item) >= 3: mode = str(item[2])
                        if d is None: continue
                        try:
                            d_label, _, _, _ = parse_isotopnavn(str(d))
                            d = d_label
                        except Exception:
                            d = str(d)
                        try:
                            br = float(br) if br is not None else None
                        except Exception:
                            br = None
                        norm.append({"daughter": d, "mode": str(mode), "bf": br})

                    if not norm:
                        lines.append(f"  {lab}: (ingen parsebare grener)")
                        continue

                    # Skriv linjer + bygg ny frontier
                    for g in norm:
                        d = g["daughter"]; mode = g["mode"]; b = g["bf"]
                        bf_new = bf * (b if (b is not None) else 1.0)
                        b_txt = f"{b*100:.2f} %" if b is not None else "ukjent BF"
                        lines.append(f"  {lab} --{mode}/{b_txt}--> {d}")
                        new_frontier.append((d, bf_new, mode))
                except Exception as ex:
                    lines.append(f"  {lab}: feil ({ex})")
            frontier = new_frontier
            lines.append("")  # tom linje mellom steg
            if not frontier: break
        return "\n".join(lines)

    # ------------------- Kjede: manuell -------------------
    def _bygg_kjede_manuell(self):
        self.txt_kjede_man.config(state="normal")
        self.txt_kjede_man.delete("1.0", "end")
        try:
            start_raw = (self.entry_kjede_start_manual.get() or "").strip()
            label, sym, A, is_m = parse_isotopnavn(start_raw)
            Z = SYMBOL_TO_Z[sym]
            seq_raw = (self.entry_kjede_seq.get() or "").strip()
            seq = [s.strip().lower() for s in re.split(r"[,\s;]+", seq_raw) if s.strip()]
            if not seq:
                raise ValueError("Tom sekvens.")
            curr_A, curr_Z = A, Z
            lines = [f"Start: {sym}-{A}"]
            for i, token in enumerate(seq, 1):
                if token in ["α","alfa","alpha","a","alpfa"]: 
                    isotop, fork = do_alpha(curr_A, curr_Z)
                    curr_A, curr_Z = curr_A - 4, curr_Z - 2
                    lines.append(f"Steg {i}: α -> {isotop}")
                elif token in ["β-","beta-","beta−","b-","bminus","e-"]:
                    isotop, fork = do_beta_minus(curr_A, curr_Z)
                    curr_A, curr_Z = curr_A, curr_Z + 1
                    lines.append(f"Steg {i}: β- -> {isotop}")
                elif token in ["β+","beta+","b+","bplus","e+"]:
                    isotop, fork = do_beta_plus(curr_A, curr_Z)
                    curr_A, curr_Z = curr_A, curr_Z - 1
                    lines.append(f"Steg {i}: β+ -> {isotop}")
                elif token in ["γ","gamma","g"]:
                    isotop, fork = do_gamma(curr_A, curr_Z)
                    lines.append(f"Steg {i}: γ -> {isotop}")
                else:
                    raise ValueError(f"Ukjent henfallstoken: '{token}'.")
            lines.append(f"Slutt: {make_isotope(curr_Z, curr_A)}")
            self.txt_kjede_man.insert("1.0", "\n".join(lines))
        except Exception as ex:
            self.txt_kjede_man.insert("1.0", f"Kunne ikke bygge manuell kjede.\nDetaljer: {ex}")
        finally:
            self.txt_kjede_man.config(state="disabled")

    # ------------------- Formatering -------------------
    @staticmethod
    def _format_mass(m_g: float) -> str:
        m = float(m_g)
        if m >= 1_000_000: val, unit = (m / 1_000_000), "tonn"
        elif m >= 1000:    val, unit = (m / 1000), "kg"
        elif m >= 1:       val, unit = m, "g"
        elif m >= 1e-3:    val, unit = (m * 1e3), "mg"
        elif m >= 1e-6:    val, unit = (m * 1e6), "µg"
        else:              val, unit = (m * 1e9), "ng"
        if val >= 100: s = f"{val:,.0f}"
        elif val >= 10: s = f"{val:,.1f}"
        else: s = f"{val:,.2f}"
        s = s.replace(",", "X").replace(".", ",").replace("X", "."); return f"{s} {unit}"

    @staticmethod
    def _fmt_pct(x: float) -> str:
        if abs(x) >= 100: s = f"{x:,.0f}"
        elif abs(x) >= 10: s = f"{x:,.1f}"
        else: s = f"{x:,.2f}"
        return s.replace(",", "X").replace(".", ",").replace("X", ".")

    @staticmethod
    def _fmt_smart(x: float) -> str:
        ax = abs(x)
        if ax >= 100: s = f"{x:,.0f}"
        elif ax >= 10: s = f"{x:,.2f}"
        elif ax >= 1: s = f"{x:,.3f}"
        elif ax >= 1e-3: s = f"{x:,.4f}"
        else: s = f"{x:.4e}"
        return s.replace(",", "X").replace(".", ",").replace("X", ".")

    @staticmethod
    def _format_bq(bq: float) -> str:
        """Formater aktivitet i Bq med SI-prefiks."""
        x = float(bq)
        absx = abs(x)
        if absx >= 1e12: val, unit = x/1e12, "TBq"
        elif absx >= 1e9: val, unit = x/1e9, "GBq"
        elif absx >= 1e6: val, unit = x/1e6, "MBq"
        elif absx >= 1e3: val, unit = x/1e3, "kBq"
        elif absx >= 1: val, unit = x, "Bq"
        elif absx >= 1e-3: val, unit = x*1e3, "mBq"
        elif absx >= 1e-6: val, unit = x*1e6, "µBq"
        else: val, unit = x*1e9, "nBq"
        if abs(val) >= 100: s = f"{val:,.0f}"
        elif abs(val) >= 10: s = f"{val:,.1f}"
        else: s = f"{val:,.2f}"
        s = s.replace(",", "X").replace(".", ",").replace("X", ".")
        return f"{s} {unit}"

# ---------------------------------------------------
# Main
# ---------------------------------------------------
if __name__ == "__main__":
    root = tk.Tk()
    app = FysikkmesterApp(root)
    root.mainloop()

In [ ]:
# Halveringstid
#Funksjon som gir hvor mange prosent som er igjen av stoffet etter t tid.
#Be om verdier for t og h.
t = float(input("Hvor lang tid har gått? "))
h = float(input("Hva er halveringstida? "))
 
#Beregn hvor mange prosent som gjenstår av massen.
p = 0.5**(t/h)*100
 
#Skriv ut svaret.
print("Det gjenstår", p, "% av massen som ennå ikke har sendt ut stråling.")

In [ ]:
#Funksjon som gir hvor mange prosent som er igjen av stoffet etter t tid.
def p(t,h):
    return  0.5**(t/h)*100
 
#Be om verdier for t og h.
t = float(input("Hvor lang tid har gått? "))
h = float(input("Hva er halveringstida? "))
 
#Skriv ut p(t,h).
print(p(t,h), "% gjenstår av opprinnelig mengde.")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
#Funksjon som gir hvor mange prosent som er igjen av stoffet.
def p(t,h):
    return  0.5**(t/h)*100
 
#Be om verdier for t og h.
t = np.linspace(0, float(input("Hvor lang tid skal gå? ")), 1000)
h = float(input("Hva er halveringstiden? "))
 
#Lage graf.
fig = plt.figure()
ax1 = fig.add_subplot(111)
ax1.grid(True)
 
x = t
y = p(t,h)
 
ax1.plot(x,y)
plt.show()

In [ ]:
#Fasit:
import numpy as np
import matplotlib.pyplot as plt
#Funksjon som gir hvor mange prosent som er igjen av stoffet etter t tid.
def p(t,h):
    return  2*0.5**(t/h)
 
#Be om verdier for t og h.
t = np.linspace(0, float(input("Hvor lang tid skal gå? ")), 1000)
h = float(input("Hva er halveringstida? "))
 
#Lag graf.
fig = plt.figure()
ax1 = fig.add_subplot(111)
ax1.grid(True)
 
x = t
y = p(t,h)
 
ax1.plot(x,y)
plt.show()

In [ ]:
#Fasit på ekstraoppgave: Hvor mange kilo startet hen med?.
import numpy as np
import matplotlib.pyplot as plt
#Funksjon som gir hvor mange prosent som er igjen av stoffet etter t tid.
def p(t,h,m):
    return  m*0.5**(t/h)
 
#Be om verdier for t og h.
m = float(input("Hvor mange kilo av grunnstoffet starter du med? "))
t = np.linspace(0, float(input("Hvor lang tid skal gå? ")), 1000)
h = float(input("Hva er halveringstida? "))
 
#Lag graf.
fig = plt.figure()
ax1 = fig.add_subplot(111)
ax1.grid(True)
 
x = t
y = p(t,h,m)
 
ax1.plot(x,y)
plt.show()

In [ ]:
#Fasit:
import numpy as np
 
#Funksjon for halveringstida.
def h(t,p):
    return  (np.log(0.5)/np.log(p/100))*t
 
#Be om verdier for t og p.
t = float(input("Hvor lang tid har gått? "))
p = float(input("Hvor mange prosent gjenstår? "))
 
#Skriv ut h(t,p).
print("Halveringstida er ", h(t,p))

In [ ]:
# Fasit: aldersbestemmelse med karbon-14
import numpy as np
 
#funksjon for å finne alder.
def t(p):
    return (np.log(p/100)/np.log(1/2))*5730
 
#Be om verdi for prosent
p = float(input("Hvor mange prosent gjenstår? "))
 
#Skriv ut hvor gammelt skjellettet er.
print("Skjellettet er ","{:.0f}".format(t(p)),"år gammelt.")